In [1]:
# Load packages
import torch
import random
import numpy as np
import pandas as pd

# Load tgt and src files
path_src_train = '../data/MIT_reagents_class/src-train.txt'
path_tgt_train = '../data/MIT_reagents_class/tgt-train.txt'
path_src_val = '../data/MIT_reagents_class/src-val.txt'
path_tgt_val = '../data/MIT_reagents_class/tgt-val.txt'
path_save_model = '../available_models/MIT_reagents_class/'
src_train = pd.read_csv(path_src_train, sep='\t', header=None).values.flatten().tolist()
tgt_train = pd.read_csv(path_tgt_train, sep='\t', header=None).values.flatten().tolist()
src_val = pd.read_csv(path_src_val, sep='\t', header=None).values.flatten().tolist()
tgt_val = pd.read_csv(path_tgt_val, sep='\t', header=None).values.flatten().tolist()

In [2]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

vocab = build_vocab_from_iterator(src_train)
l_names = sorted(set(tgt_train))
labels = {label:i for i, label in enumerate(l_names)}
# ocurrences   = [tgt_train.count(n) for n in labels]

1226084lines [00:09, 127762.54lines/s]


In [ ]:
train_iter = ((label[]))

In [3]:
import re

# From SMILES to tokens
def smi_tokenizer(smi):
    """
    Tokenize a SMILES molecule or reaction
    """
    pattern =  "(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
    regex = re.compile(pattern)
    tokens = [token for token in regex.findall(smi)]
    assert smi == ''.join(tokens)
    return ' '.join(tokens)

tokenizer = lambda x: ' . '.join([smi_tokenizer(n) for n in x.split('.')])
text_pipeline = lambda x: [vocab[n] for n in x.split()] # use tokenizer if reactions are not tokenized
label_pipeline = lambda x: labels[x]

In [4]:
from torch.utils.data import DataLoader
device = torch.device("cuda:2")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

dataloader = DataLoader(src_train, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [5]:
from torch import nn

class ReactionClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(ReactionClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [8]:
num_class = len(labels)
vocab_size = len(vocab)
emsize = 64
model = ReactionClassificationModel(vocab_size, emsize, num_class).to(device)

In [10]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [12]:
from torch.utils.data.dataset import random_split

# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_dataloader = DataLoader(src_train, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(src_val, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)

In [17]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

ValueError: too many values to unpack (expected 2)